### LDA + Word2Vec 사용시
#### Word2Vec과 LDA를 통합해 사용하면 Word2Vec의 문맥적 관계뿐만 아니라 LDA의 문서와 주제간의 관계를 얻을 수 있음
    (참고문헌 : WANG, Zhibo; MA, Long; ZHANG, Yanqing. A hybrid document feature extraction method using latent Dirichlet allocation and word2vec. In: Data Science in Cyberspace (DSC), IEEE International Conference on. IEEE, 2016. p. 98-103.)

In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv(r"C:\Users\User\Desktop\game_word2vec\review_data_all.csv",encoding='cp949')
data.head(5)

,id,kind_of_games,user_name,review,score,thumb_count
0,0,Counter-Strike,retr0,Counter-strike(1.0 - 1.6 not source) is one of...,10,26//26
1,1,Counter-Strike,sinadoom,After all these years this shooter remains to ...,8,6//6
2,2,Counter-Strike,Gratson,The greatest multiplayer shooter experience I'...,10,3//3
3,3,Counter-Strike,PevMaster,The year was 2000. Half-Life had already taken...,10,3//3
4,4,Counter-Strike,rp4prez,One of the best games of all time... Never for...,10,3//3


In [ ]:
names = set(data['kind_of_games'])

names_words=[]
for x in names:
    names_words += [word.lower() for word in x.split()]

cnt = collections.Counter()
for x in names_words:
    cnt[x] += 1

* DATA 전처리

In [ ]:
review = [re for re in data['review']]

# stemming
stemmer = SnowballStemmer("english", ignore_stopwords=True)

stem_review = []
for ii, words in enumerate(review):
    print("stemmer--> ", ii, "/", len(review))
    stem_review.append(stemmer.stem(words))

stem_review = np.array(stem_review)
def eli_stop(string, stop_word, rlc = ''):
    for sw in stop_word:
        string= string.replace(sw, rlc)
    return string


eli = ['nothing','better','way','nice','new','great','awesome','more','lot', 'many', 'other', 'time', 'same', 'everything', 'hours', 'things', 'bit', 'most','much',"good", "bad","!", "...", ")", "(", "/", ".", ",", "?", "-", "''", "``", "'d", ":", ";", "***", "*", "%", "$", "@", "#", "&", "+", "~", "'s", "n't", "'m", "game","'"]
eli_review_list = [eli_stop(str(rm),eli) for rm in stem_review]

sens = [nltk.tokenize.sent_tokenize(eli_list) for eli_list in eli_review_list]

tokens = []
for i in range(0, len(sens)):
    token = []
    for j in range(0, len(sens[i])):
        token += nltk.tokenize.word_tokenize(sens[i][j])
    tokens.append(list(token))

pos = []
for t in tokens:
    pos_tokens = [token for token, pos in nltk.pos_tag(t) if pos.startswith('NN')|pos.startswith('JJ')]
    pos.append(pos_tokens)

collections.OrderedDict(sorted(cnt.items(), key=lambda t: t[1]))

np.shape(pos)
len(np.unique(pos))

# removal stopwords
stop = nltk.corpus.stopwords.words('english')

additionalstop = ['lot', 'many', 'other', 'time', 'same', 'everything', 'hours', 'things', 'bit', 'most','much','more','good','bad','sure','fun','great','game','make', 'un', 'es', 'juego', 'la', 'el', 'con', 'lo', 'los', 'para', 'una', 'si', 'se', 'por', 'le','rollerskates']
stop += additionalstop
stop += names_words

removed_words = [a for a in pos if a not in stop]
ree = [a for i in removed_words for a in i]
removed_voc = list(set(ree))

len(set(removed_voc))


one_list = []
for i in ree:
    one_list.append([i[l] for l in range(len(i)) if len(i[l])<2])

clean_words = [a for a in removed_words if a not in one_list]

final_words = []
for a_line in removed_words:
        final_words.append([a_line[a_num] for a_num in range(len(a_line)) if len(str(a_line[a_num]))>1])

final_words_list = [a for i in final_words for a in i]


* matrix 생성

In [ ]:
# Word2Vec
from gensim.models import Word2Vec

embedding_model = Word2Vec(final_words, size=100, window = 3,min_count=1)

In [ ]:
# Word2Vec + LDA matrix

# word2vec mat
em_mat=[]
row_name = []
for ii, list_words in enumerate(final_words):
    for i in list_words:
        if i in embedding_model:
            em_mat.append(embedding_model[str(i)])
            row_name.append(i)
    print('turn-->', ii, "/", len(final_words))

mat_list = pd.DataFrame(em_mat,index=[row_name])

In [6]:
mat_list = pd.read_csv(r"C:\Users\User\PycharmProjects\word2vec\word2vec_word.csv",encoding='cp949')
mat_list.head(3)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,source,0.618804,0.145858,0.523411,0.520021,-0.173712,0.307795,0.899815,-0.181912,0.403240,...,0.105109,0.186894,0.321012,0.387029,-0.191796,-0.284091,-0.947530,0.204489,0.359828,0.118231
1,greatest,-0.063686,0.364259,-0.309629,1.098529,-0.461379,0.411075,0.584363,-0.409022,0.423578,...,0.340206,0.212385,0.054433,-0.162662,-0.898852,-0.235662,-0.270714,-0.339825,0.161816,-0.001008
2,esport,-0.017050,-0.032554,-0.009539,0.070001,-0.023072,0.016093,0.097661,-0.044066,0.129004,...,0.005854,0.017711,0.000467,0.035074,-0.103017,-0.033291,-0.054472,-0.055531,0.057393,0.043367


In [ ]:
countvec = CountVectorizer(analyzer='word',vocabulary=list(set(final_words_list)))
count = countvec.fit_transform(final_words_list)
topic_vocab=countvec.get_feature_names()
titles = names

# LDA modeling
model = lda.LDA(n_topics=12, n_iter=500, random_state=6, alpha=0.1)
model.fit(count)
topic_word = np.array(topic_vocab)[np.argsort(model.topic_word_)][:,:-1]

# lda percent
word_topic = np.array(np.argsort(model.topic_word_))[:,::-1]
np.shape(word_topic)

![vector](https://github.com/Oh-Yoojin/Review-Analysis/blob/master/pictures/vector.png)

In [ ]:
# combine model - topic vector --------------------
lda_mat = []
for ii, wor in enumerate(topic_word):
    print("turn--> ", ii, '/', len(topic_word))
    wor_vec = []
    for wo in wor:
        wor_vec.append(embedding_model[wo])
    lda_mat.append(wor_vec)


In [ ]:
topic_range=range(12)

theta_i = []
for n, i in enumerate(topic_range):
    print("turn--> ", n, "/", len(topic_range))
    topic_theta = [topic/sum(word_topic[i]) for topic in word_topic[i]]
    theta_i.append(topic_theta)

topic_vectors = []
for a in range(12):
    cal_theta = [lda_mat[a][i]*theta_i[a][i] for i in range(len(lda_mat[a]))]
    topic_vectors.append(cal_theta)

topic_vec_final = [np.array(topic_vectors[i]).sum(axis=0) for i in topic_range]

In [ ]:
# comb modeling - document vector ---------------------------------

fi_sent_word = pd.DataFrame(final_words)

game = np.array(data['kind_of_games'])
len(names)
data.columns[:]

review_data = data['kind_of_games']
review_data = pd.DataFrame(review_data)

total_docu_vec = []
for n, docu in enumerate(final_words):
    print('trun--> ', n, '/', len(final_words))
    docu_vector = []
    for wo in docu:
        docu_vector.append(embedding_model[wo])
    total_docu_vec.append(docu_vector)


game_name = list(names)
game = np.array(data['kind_of_games'])

game_re_vec = []
for n, na in enumerate(game_name):
    print("turn--> ", n, "/", len(names))
    what_game = []
    for ga in range(len(game)):
        if na == game[ga]:
            what_game.append(total_docu_vec[ga])
    game_re_vec.append(what_game)

In [ ]:

sum_mat = []
for line in range(len(game_re_vec)):
    print('turn--> ', line, '/', len(game_re_vec))
    game_line = [a for i in game_re_vec[line] for a in i]
    sum_line = []
    for num in range(100):
        tmp = [sum(game_line[n][num] for n in range(len(game_line)))/len(game_line)]
        sum_line.append(tmp)
    sum_mat.append(sum_line)

In [ ]:
# calculate distance ------------------------------------

dist_mat = []
for p in range(len(sum_mat)):
    print("turn--> ", p, '/', len(sum_mat))
    x_vec = [a for i in sum_mat[p] for a in i]
    dist = [euclidean_distances(x_vec,topic_vector[n]).tolist() for n in range(len(topic_vector))]
    bb = [a for i in dist for a in i]
    dist_mat.append([a for i in bb for a in i])

In [7]:
distance = pd.read_csv(r"C:\Users\User\PycharmProjects\word2vec\distance_mat.csv")
distance.head(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11
0,0,4.507636,4.508133,4.506932,4.507457,4.505924,4.505517,4.506538,4.506559,4.505094,4.507807,4.504379,4.508221
1,1,3.873360,3.873759,3.872551,3.873103,3.871565,3.871041,3.871916,3.872049,3.870439,3.873375,3.870060,3.873853
2,2,3.962925,3.963367,3.962256,3.962882,3.961172,3.960739,3.961619,3.961596,3.960179,3.963042,3.959623,3.963456
3,3,3.764286,3.764648,3.763470,3.764056,3.762398,3.762029,3.762873,3.762921,3.761468,3.764369,3.760894,3.764794
4,4,3.687355,3.687870,3.686677,3.687280,3.685580,3.685187,3.686110,3.686037,3.684571,3.687498,3.683966,3.687880


In [ ]:
# select topic ----------------------------------------

selected_topic = [np.argsort(dist_to[n])[:5] for n in range(len(dist_to))]

In [8]:
selected_topic = pd.read_csv(r"C:\Users\User\PycharmProjects\word2vec\dist_topic.csv")
selected_topic.head(5)

,Unnamed: 0,0,1,2,3,4
0,0,10,8,5,4,6
1,1,10,8,5,4,6
2,2,10,8,5,4,7
3,3,10,8,5,4,6
4,4,10,8,5,4,7
